# Introduction to Machine Learning

Hello! This notebook is intended to be an introduction to machine learning. The model we will create won't be super advanced, but it will serve as a good basis for you to explore more on your own.

## The Data

Before we get started, I should introduce the data we will be working with. The Boston Housing dataset is a very common beginner dataset (So much so, that it's included in the library we will be using!). 

This data was collected by the US Census Bureau in 1978 in the suburbs surrounding Boston, Massachusetts. It includes 14 statistics for 506 suburbs, including:
* Per capita crime rate by town
* Average number of rooms per dwelling
* Proportion of owner-occupied units built prior to 1940

Let's go ahead and load this dataset into our environment. We are using the sklearn library in this tutorial as it provides an easy way to access all of the information we need.

In [264]:
from sklearn.datasets import load_boston #Import the Boston dataset loader function from the datasets module.

dataset = load_boston() # Loads a dictionary full of the dataset into our variable.


Our `dataset` variable now includes some useful information:


In [265]:
print(dataset.keys())

['filename', 'data', 'target', 'DESCR', 'feature_names']


`'filename'` just tells us where the dataset is stored on our computer.

`'data'` contains all the 506 examples and their values.

`'target'` is the value we will try to predict for all of the 506 examples.

`'DESCR'` is more information about the dataset which you should certainly read.

`'feature_names'` tells you to what each of the columns in `'data'` corresponds (the statistics I mentioned above). 

In our case, the 'target' we will be trying to predict is the median household value in \$1000's.

## But How?
The algorithm we will be looking at today is called linear regression. If you have ever used the "line of best fit" feature in Microsoft Excel, you are already familiar. In fact, you could probably do what we're about to do in Excel, but where's the fun in that?

### The Hypothesis Function
We are all familiar with the basic formula of a line:

\begin{aligned}
y = mx+b \\
\end{aligned}

In this case, we have our y (the cost of the house), and our x (the 13 other statistics), and now we want to find the m that will, given any _unseen_ (as in, nothing we already have in our dataset) x values, give us the correct y value.

In machine learning terms,this formula is called our _hypothesis function_. Typically, this will be written in the form:

\begin{aligned}
h_\theta(x) = \theta x \\
\end{aligned}

In our case, it would actually look more like this:

\begin{aligned}
h_\theta(x) = \theta_0x_0 + \theta_1 x_1 + \dotsb + \theta_0 x_{12} \\
\end{aligned}

Since we have 13 x's.

### The Cost Function
So now that we have our function defined, how do we "grade" our prediction? That's where the cost function comes into play! Now, there are many cost functions out there and which one you use depends very much on the problem you are trying to solve, and the later steps in this tutorial. That discussion, however, is outside the scope of this tutorial.

In our case, the cost function that we will be using is called the Mean Squared Error. That function looks like this:
\begin{aligned}
J(\theta) = \frac{1}{2m}\sum_{i=0}^{m} (y_i - h_{i\theta}(x))^2 \\
\end{aligned}
Where $h_{i\theta}(x)$ is what we predicted for the $i^{th}$ example and $y_i$ is the correct value. As you can see, this simply takes the average of the squared difference between the real value and the predicted value. The $\frac{1}{2}$ part just makes the math nice when you take the first derivative of the equation (we'll talk about that later).

### Gradient Descent
Okay, so now that we have the hypothesis function, the correct values, and a way to quantify our error, what next? We need to find a way to improve our guess for $\theta$ using our cost function. The way we are going to do that is called gradient descent.

Gradient descent will use the cost function we defined above to gradually change the value of $\theta$ until the cost is zero, or as close to it as we can get. The formula is as follows:

\begin{aligned}
\theta_j := \theta_j - \alpha\frac{\partial}{\partial \theta_j}J(\theta)\\
\end{aligned}

So, for every $\theta$ value, subtract from it's current value the derivate of the cost function with respect to that $\theta$. $\alpha$ is the rate by which we change that value, also called the learning rate. This typically has a low value like .01.

I know the formula looks scary, but the intuition behind it is rather simple: update each $\theta$ by the amount of change it affects in the change (or slope) of the cost function (scaled by how fast we want the the whole process to go).

## The Real Fun
Enough boring math, it's time to try this out!

### The Model
So the first step is to import the libraries that we will need. We will be using keras in this case since it provides (in my opinion) the easiest interface for creating models.

In [266]:
''' I wouldn't pay too much attention to what these do for now, they are just libraries specific to keras
  that we will need
'''
from keras.models import Sequential
from keras.layers import Input, Dense

In [267]:
model = Sequential() # Create our model
model.add(Dense(1, activation="linear",input_shape=(4,))) #Create the first and only layer of our model
model.compile(optimizer="SGD", loss="mean_squared_error")

Let me explain the last two lines a little. Since Keras is made for neural networks, we have to do some ugly stuff to make it do linear regression. All this does is create a single "neuron" using the "linear" function (our hypothesis function), and tell it that our input will be 4 features (the reason I chose these features specifically is outside the scope of this tutorial). 

Then in the next line, we tell it we want to learn using SGD (Stochasic Gradient Descent, our gradient descent algorithm above). We also tell it we want to use our Mean Squared Error function.

### The Data
So before we try to run our model, we should do some data preprocessing. What we are going to do is normalize the features we have. We do this to ensure our model trains well. If we leave the data as it is, the changes our gradient descent makes to our weights can be drastic or otherwise unhelpful. Let's consider the first sample:

In [268]:
data = dataset['data'][:,[0,5,9,12]] # Get the data for all the samples, but only the 4 features I have chosen.
print(data[0])

[6.320e-03 6.575e+00 2.960e+02 4.980e+00]


As you can see, some values are very small and others are rather large. That means small steps for the large features will mean huge steps for the small variables, and we'll potentially never reach the minimum!

We will be using numpy for this part.

In [269]:
import numpy as np
data = np.array(data) # Convert train_data to a numpy array for these operations
average = np.average(data) # Calculate the average of every sample
std = np.std(data) # Calculate the standard deviation of values for every sample

data = (data - average) / (std)

Now, as you can see, the distribution of values is much smaller:

In [270]:
print(data[0])

[-0.55807052 -0.52403059  0.97581526 -0.53229613]


Now let's split up the data so that we reserve 20% of the dataset for testing our model.

In [271]:
split_index = int(len(data) * .20)
test_data = data[:split_index]
train_data = data[split_index:]

test_labels = dataset['target'][:split_index]
train_labels = dataset['target'][split_index:]

Now, you should never split your data the way we just did. The reason for that is: if there is any trend related to the order of your data, your model will learn this instead and not be accurate for new data. You should typically randomly sample the data for the splits, but in our case it will be fine.

### Putting it all together
Now is the moment of truth. Let's train our model and test it.

In [272]:
model.fit(x=train_data, y=train_labels, epochs=100)

Epoch 1/100
405/405 [==============================] - 0s 785us/step - loss: 363.7975
Epoch 2/100
405/405 [==============================] - 0s 31us/step - loss: 207.2077
Epoch 3/100
405/405 [==============================] - 0s 37us/step - loss: 180.9793
Epoch 4/100
405/405 [==============================] - 0s 54us/step - loss: 164.1028
Epoch 5/100
405/405 [==============================] - 0s 43us/step - loss: 150.6410


405/405 [==============================] - 0s 56us/step - loss: 66.5285
Epoch 98/100
405/405 [==============================] - 0s 51us/step - loss: 66.5634
Epoch 99/100
405/405 [==============================] - 0s 53us/step - loss: 66.5094
Epoch 100/100
405/405 [==============================] - 0s 58us/step - loss: 66.5008


In [273]:
from sklearn.metrics import mean_squared_error # We'll use this to calculate the mean squared error for us
test_predictions = model.predict(test_data) # Have our model make predictions on new, unseen data
error = mean_squared_error(test_labels, test_predictions) # Calculate our error rate across the test set
print(error)

50.56089221973576


## Conclusion
As you can see from the output of the last cell, the more iterations we ran, the lower our loss became. You can play around and see how many epochs it takes for it to finally reach zero (or if it ever even does!), or try different loss functions, features, or even normalization methods. I hope you learned something from this tutorial, and if you have any recommendations on how to make this better or any errors you might've noticed, PLEASE reach out to me.